In [2]:
import numpy as np
from tqdm import tqdm

import tensorflow as tf
import keras
from keras.preprocessing import timeseries_dataset_from_array
from keras.callbacks import CSVLogger
from sklearn.metrics import classification_report

import modules.data as data
from modules.utilites import DataClass

In [3]:
# Model parametrs
class Input_pars(DataClass):
    seq_len = 100


class CN_pars(DataClass):
    n_filters = 14
    dilation_steps = 4  # dilation = 2**dilation_step


class AN_pars(DataClass):
    attention_heads = 3
    blocks = 2
    share_weights = False


class FF_pars(DataClass):
    dropout_rate = 0.1


class Optimaser_pars(DataClass):
    lr = 0.0001
    adam_beta1 = 0.9
    adam_beta2 = 0.999


class Trainig_pars(DataClass):
    shuffle = True
    batch_size = 512
    epochs = 150


class Full_pars(DataClass):
    seq_len = 100
    cn = CN_pars()
    an = AN_pars()
    ff = FF_pars()
    optimazer = Optimaser_pars()
    training = Trainig_pars()


pars = Full_pars()

In [4]:
# i= keras.Input(shape=(20,20))
# x= keras.layers.Reshape((20,20,1))(i)

# x= keras.layers.Conv2D(
#     kernel_size=(1,2),
#     filters=9,
#     padding='same',
#     # input_shape=(None, 20, 20),
# )(x)

# model = keras.Model(inputs=i, outputs=x)
# model.summary()

In [5]:
# # Model
# inputs = blocks.input_block(pars.seq_len)
# x= inputs


# x = blocks.cnn_block(
#     input_layer=x,
#     filters=pars.cn.n_filters,
#     dilation_steps=pars.cn.dilation_steps,
# )

# # x= keras.layers.Reshape(target_shape=(100,14,1))(x)
# # x = LobFeatures.lob_inception(x)
# # x= LobFeatures.lob_cnn(x)


# x = blocks.norm_block(input_layer=x)

# x = blocks.positional_encoder_block(input_layer=x)


# x = blocks.transformer_block(
#     input_layer=x,
#     n_blocks=pars.an.blocks,
#     n_heads=pars.an.attention_heads,
#     share_weights=pars.an.share_weights,
# )


# x = blocks.ffn_block(
#     input_layer=x,
#     dropout_rate=pars.ff.dropout_rate,
# )


# model = keras.Model(inputs=inputs, outputs=x)
# model.summary(
#     line_length=100,
#     expand_nested=True,
# )

In [6]:
((x_train, y_train), (x_val, y_val), (x_test, y_test)) =(
# data.load_dataset(horizon=4)
data.load_saved_data()
)

In [7]:
# data.save_data(name= 'train',x= x_train,y=y_train)
# data.save_data(name= 'val',x= x_val,y=y_val)
# data.save_data(name= 'test',x= x_test,y=y_test)

In [8]:
# Dataset
ds_train = data.build_dataset(
    x=x_train,
    y=y_train,
    batch_size=pars.training.batch_size,
    seq_len=pars.seq_len,
)

ds_val = data.build_dataset(
    x=x_val,
    y=y_val,
    batch_size=pars.training.batch_size,
    seq_len=pars.seq_len,
)

ds_test = data.build_dataset(
    x=x_test,
    y=y_test,
    batch_size=pars.training.batch_size,
    seq_len=pars.seq_len,
)

print(
    f'Train x: {str(x_train.shape): <15} - y: {str(y_train.shape): <15}'+\
    f' - tensor: {len(ds_train)} - {list(ds_train.element_spec[0].shape)}',
    f'Val   x: {str(x_val.shape): <15} - y: {str(y_val.shape): <15}'+\
    f' - tensor: {len(ds_val)} - {list(ds_val.element_spec[0].shape)}',
    f'Test  x: {str(x_test.shape): <15} - y: {str(y_test.shape): <15}'+\
    f' - tensor: {len(ds_test)} - {list(ds_test.element_spec[0].shape)}',
    sep='\n',
)

Train x: (203799, 40)    - y: (203799,)       - tensor: 398 - [None, 100, 40]
Val   x: (50949, 40)     - y: (50949,)        - tensor: 100 - [None, 100, 40]
Test  x: (139586, 40)    - y: (139586,)       - tensor: 273 - [None, 100, 40]


In [9]:
from models import base_model,model2


In [10]:
# CompiLe
# model.compile(
#     keras.optimizers.Adam(
#         learning_rate=pars.optimazer.lr,
#         beta_1=pars.optimazer.adam_beta1,
#         beta_2=pars.optimazer.adam_beta2,
#         name="Adam",
#     ),
#     loss=keras.losses.SparseCategoricalCrossentropy(),
#     metrics=[
#         keras.metrics.CategoricalAccuracy(name='accurancy'),
#         keras.metrics.SparseCategoricalAccuracy(name='sparce_accurancy'),
#     ],
# )

In [11]:
model = base_model.build_model()

In [1]:
from keras_tuner.tuners import GridSearch


In [12]:
adam= keras.optimizers.Adam(
        learning_rate=0.1,
        beta_1=0.99,
        beta_2=0.9,
        name="Adam",
    )

In [18]:
pars.Info_expanded

{'seq_len': 100,
 'cn__n_filters': 14,
 'cn__dilation_steps': 4,
 'an__attention_heads': 3,
 'an__blocks': 2,
 'an__share_weights': False,
 'ff__dropout_rate': 0.1,
 'optimazer__lr': 0.0001,
 'optimazer__adam_beta1': 0.9,
 'optimazer__adam_beta2': 0.999,
 'training__shuffle': True,
 'training__batch_size': 512,
 'training__epochs': 150}

In [13]:
gs = GridSearch(
    hypermodel=model,
    hyperparameters={
        'seq_len': 100,
        'cn__n_filters': 14,
        'cn__dilation_steps': 4,
        'an__attention_heads': 3,
        'an__blocks': 2,
        'an__share_weights': False,
        'ff__dropout_rate': 0.1,
    },
    
)


AttributeError: 'dict' object has no attribute 'space'

In [ ]:
gs.fit(ds_train)

In [13]:
# Train
model.fit(
    ds_train,
    # epochs=pars.training.epochs,
    epochs=10,
    validation_data=ds_val,
    # callbacks=[
    #     CSVLogger(r'logs/model1/log.csv', append=True, separator=';')
    # ]
)

Epoch 1/10
  9/398 [..............................] - ETA: 6:27 - loss: 2.6096 - accurancy: 0.5788 - sparce_accurancy: 0.4206

KeyboardInterrupt: 